# 🏇 JRA 基本情報スクレイピング (Stage 1/2) v2

## 📊 取得データ
- **32カラム**: 日付、会場、レース番号、レース名、重賞、コースタイプ、距離、回り、天候、馬場状態、着順、枠、馬番、馬名、性齢、斤量、騎手、タイム、着差、人気、単勝オッズ、後3F、corner_1, corner_2, corner_3, corner_4, 厩舎, 調教師, 馬体重, 増減, race_id, horse_id

## ✅ 改善点
- メタデータ抽出を確実に
- カラムズレを完全に防止
- データ検証を実装

## 🚀 次のステップ
Stage 2で馬履歴・血統データを取得

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🛡️ Keep-Alive (アイドルタイムアウト回避)
from IPython.display import display, Javascript

display(Javascript('''
function ClickConnect(){
    console.log("Keep-alive: Working");
    var buttons = document.querySelectorAll("colab-connect-button");
    buttons.forEach(function(btn){
        btn.click();
    });
}
setInterval(ClickConnect, 60000);
console.log("Keep-alive script started - clicks every 60 seconds");
'''))

print("✅ Keep-alive activated")

In [ ]:
# Settings
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
TARGET_CSV = 'database_basic.csv'
MASTER_ID_CSV = 'race_ids.csv'

In [ ]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import gc
from tqdm.auto import tqdm
from datetime import datetime

print("✅ Libraries loaded")

In [ ]:
# メタデータ抽出関数(完全版 - 11/11カラム取得)

def extract_metadata(soup, url):
    """全メタデータを確実に抽出"""
    metadata = {
        '日付': '', '会場': '', 'レース番号': '', 'レース名': '', '重賞': '',
        'コースタイプ': '', '距離': '', '回り': '', '天候': '', '馬場状態': '', 'race_id': ''
    }
    
    try:
        title = soup.title.text if soup.title else ""
        full_text = soup.text
        
        if title:
            # レース名
            race_name_match = re.search(r'^([^|]+)', title)
            if race_name_match:
                race_name_full = race_name_match.group(1).strip()
                race_name = re.sub(r'\s*(結果|払戻|払い戻し).*$', '', race_name_full).strip()
                metadata['レース名'] = race_name
                
                # 重賞判定
                if 'G1' in race_name or 'GⅠ' in race_name or 'GI' in race_name:
                    metadata['重賞'] = 'G1'
                elif 'G2' in race_name or 'GⅡ' in race_name or 'GII' in race_name:
                    metadata['重賞'] = 'G2'
                elif 'G3' in race_name or 'GⅢ' in race_name or 'GIII' in race_name:
                    metadata['重賞'] = 'G3'
            
            # 日付
            date_match = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日', title)
            if date_match:
                year = date_match.group(1)
                month = f"{int(date_match.group(2)):02d}"
                day = f"{int(date_match.group(3)):02d}"
                metadata['日付'] = f"{year}/{month}/{day}"
            
            # 会場
            venues = ['札幌', '函館', '福島', '新潟', '東京', '中山', '中京', '京都', '阪神', '小倉']
            for venue in venues:
                if venue in title:
                    metadata['会場'] = venue
                    break
            
            # レース番号
            race_num_match = re.search(r'(\d+)R', title)
            if race_num_match:
                metadata['レース番号'] = race_num_match.group(0)
        
        # コースタイプ・距離
        course_match = re.search(r'(芝|ダート|ダ|障害)\s*(\d+)m', full_text)
        if course_match:
            course_type = course_match.group(1)
            if '芝' in course_type:
                metadata['コースタイプ'] = '芝'
            elif 'ダ' in course_type:
                metadata['コースタイプ'] = 'ダート'
            elif '障' in course_type:
                metadata['コースタイプ'] = '障害'
            metadata['距離'] = course_match.group(2)
        
        # 回り
        if '右' in full_text:
            metadata['回り'] = '右'
        elif '左' in full_text:
            metadata['回り'] = '左'
        elif '直線' in full_text or '直' in full_text:
            metadata['回り'] = '直線'
        
        # 天候
        weather_match = re.search(r'天候\s*[:：]\s*(\S+)', full_text)
        if weather_match:
            metadata['天候'] = weather_match.group(1)
        
        # 馬場状態(確実に取得)
        baba_match = re.search(r'馬場\s*[:：]\s*(\S+)', full_text)
        if baba_match:
            metadata['馬場状態'] = baba_match.group(1)
        else:
            if metadata['コースタイプ'] == '芝':
                condition_match = re.search(r'芝\s*[:：]\s*(\S+)', full_text)
                if condition_match:
                    metadata['馬場状態'] = condition_match.group(1)
            elif metadata['コースタイプ'] == 'ダート':
                condition_match = re.search(r'ダート\s*[:：]\s*(\S+)', full_text)
                if condition_match:
                    metadata['馬場状態'] = condition_match.group(1)
        
        # race_id生成
        if metadata['日付'] and metadata['会場'] and metadata['レース番号']:
            year = metadata['日付'][:4]
            place_map = {
                '札幌': '01', '函館': '02', '福島': '03', '新潟': '04', '東京': '05',
                '中山': '06', '中京': '07', '京都': '08', '阪神': '09', '小倉': '10'
            }
            place_code = place_map.get(metadata['会場'], '00')
            race_num = metadata['レース番号'].replace('R', '')
            race_num_padded = f"{int(race_num):02d}"
            
            kai = '01'
            nichi = '01'
            kai_match = re.search(rf'(\d+)回{metadata["会場"]}(\d+)日', title + full_text)
            if kai_match:
                kai = f"{int(kai_match.group(1)):02d}"
                nichi = f"{int(kai_match.group(2)):02d}"
            
            metadata['race_id'] = f"{year}{place_code}{kai}{nichi}{race_num_padded}"
    
    except Exception as e:
        print(f"  ⚠️ メタデータ抽出エラー: {e}")
    
    return metadata

print("✅ Metadata extraction function loaded (Complete - 11/11 columns)")


In [ ]:
# レース結果スクレイピング関数

# 調教師名キャッシュ
TRAINER_CACHE = {}


# 騎手名キャッシュ
JOCKEY_CACHE = {}

def get_jockey_fullname(url, default_name):
    """騎手ページのリンクからフルネームを取得"""
    if not url: return default_name
    if url in JOCKEY_CACHE: return JOCKEY_CACHE[url]
    
    try:
        time.sleep(0.3)
        headers = {'User-Agent': 'Mozilla/5.0'}
        if url.startswith('/'): url = f"https://race.netkeiba.com{url}"
        
        r = requests.get(url, headers=headers, timeout=5)
        r.encoding = 'EUC-JP'
        s = BeautifulSoup(r.text, 'html.parser')
        
        title = s.title.text if s.title else ""
        m = re.search(r'^(.+?)(?:のプロフィール|の騎手成績|の調教師成績|｜)', title)
        if m:
            fullname = m.group(1).strip()
            JOCKEY_CACHE[url] = fullname
            return fullname
            
        h1 = s.find('h1')
        if h1:
             txt = h1.text.strip().split()[0]
             JOCKEY_CACHE[url] = txt
             return txt
    except:
        pass
    
    JOCKEY_CACHE[url] = default_name
    return default_name


def get_trainer_fullname(url, default_name):
    """調教師ページのリンクからフルネームを取得"""
    if not url: return default_name
    
    # キャッシュチェック
    if url in TRAINER_CACHE:
        return TRAINER_CACHE[url]
    
    try:
        time.sleep(0.3) # 負荷軽減
        headers = {'User-Agent': 'Mozilla/5.0'}
        # URLが相対パスの場合の対応
        if url.startswith('/'):
             url = f"https://race.netkeiba.com{url}"
             
        r = requests.get(url, headers=headers, timeout=5)
        r.encoding = 'EUC-JP'
        s = BeautifulSoup(r.text, 'html.parser')
        
        # タイトルから抽出: "青木孝文のプロフィール | ..."
        title = s.title.text if s.title else ""
        m = re.search(r'^(.+?)(?:のプロフィール|の騎手成績|の調教師成績|｜)', title)
        if m:
            fullname = m.group(1).strip()
            TRAINER_CACHE[url] = fullname
            return fullname
            
        # h1から抽出
        h1 = s.select_one('div.id_visual h1')
        if h1:
             txt = h1.text.strip().split()[0]
             TRAINER_CACHE[url] = txt
             return txt
             
    except Exception:
        pass
    
    # 失敗時はデフォルト(短縮名)をキャッシュして返す(再試行しない)
    TRAINER_CACHE[url] = default_name
    return default_name

def scrape_race_basic(race_id):
    """
    1レースの基本情報(27カラム: 調教師追加)を取得
    """
    url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
    
    try:
        # ページ取得
        time.sleep(0.5)
        headers = {'User-Agent': 'Mozilla/5.0'}
        resp = requests.get(url, headers=headers, timeout=15)
        
        # HTTPステータスチェック
        if resp.status_code != 200:
            print(f"  ⚠️ HTTP {resp.status_code}: {race_id}")
            return None
        
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # メタデータ抽出
        metadata = extract_metadata(soup, url)
        
        # レース結果テーブル取得
        tables = soup.find_all('table')
        result_table = None
        
        for table in tables:
            if '着順' in table.text and '馬名' in table.text:
                result_table = table
                break
        
        if not result_table:
            print(f"  ⚠️ レース結果テーブルが見つかりません: {race_id}")
            return None
        
        # 各馬のデータを取得
        rows = result_table.find_all('tr')
        race_data = []
        
        for row in rows:
            if row.find('th'):
                continue  # ヘッダー行スキップ
            
            cells = row.find_all('td')
            if len(cells) < 10:
                continue
            # --- 騎手 (Fullname) ---
            jockey_col = cells[6] if len(cells) > 6 else None
            jockey_val = ""
            if jockey_col:
                j_text = jockey_col.text.strip()
                j_link = jockey_col.find('a')
                j_url = j_link['href'] if j_link else None
                
                # Optimization: Try title attribute first
                if j_link and j_link.get('title'):
                    jockey_val = j_link.get('title')
                    if j_url: JOCKEY_CACHE[j_url] = jockey_val
                elif j_url:
                    jockey_val = get_jockey_fullname(j_url, j_text)
                else:
                    jockey_val = j_text
            
            
            # 厩舎/調教師 分離
            stable_col = cells[13] if len(cells) > 13 else None
            stable_val = ""
            trainer_val = ""
            
            if stable_col:
                raw_text = stable_col.text.strip()
                # リンク取得
                t_link = stable_col.find('a')
                t_url = t_link['href'] if t_link and 'href' in t_link.attrs else None
                
                # 分離: "美浦\n\n青木" -> ["美浦", "青木"]
                parts = [p for p in raw_text.replace('\n', ' ').split() if p.strip()]
                temp_name = ""
                
                if len(parts) >= 2:
                    stable_val = parts[0]
                    temp_name = parts[-1]
                elif len(parts) == 1:
                    # 結合されている場合 "北海道田中"
                    if raw_text.startswith("美浦") or raw_text.startswith("栗東"):
                         stable_val = raw_text[:2]
                         temp_name = raw_text[2:]
                    else:
                         temp_name = raw_text
                
                # フルネーム解決
                if t_url:
                    trainer_val = get_trainer_fullname(t_url, temp_name)
                else:
                    trainer_val = temp_name
            
            # --- 馬体重 (Split) ---
            weight_col = cells[14] if len(cells) > 14 else None
            weight_val = ""
            weight_change = ""
            if weight_col:
                w_text = weight_col.text.strip()
                m = re.search(r'(\d+)\(([\+\-\d]+)\)', w_text)
                if m:
                    weight_val = m.group(1)
                    weight_change = m.group(2).replace('+', '')
                else:
                    weight_val = w_text
            
            # 基本情報を辞書で構築
            # --- 馬体重 (Split) ---
            weight_col = cells[14] if len(cells) > 14 else None
            weight_val = ""
            weight_change = ""
            if weight_col:
                w_text = weight_col.text.strip()
                m = re.search(r'(\d+)\(([\+\-\d]+)\)', w_text)
                if m:
                    weight_val = m.group(1)
                    weight_change = m.group(2).replace('+', '')
                else:
                    weight_val = w_text
            
            horse_data = {
                '日付': metadata['日付'],
                '会場': metadata['会場'],
                'レース番号': metadata['レース番号'],
                'レース名': metadata['レース名'],
                '重賞': metadata['重賞'],
                'コースタイプ': metadata['コースタイプ'],
                '距離': metadata['距離'],
                '回り': metadata['回り'],
                '天候': metadata['天候'],
                '馬場状態': metadata['馬場状態'],
                '着順': cells[0].text.strip(),
                '枠': '',
                '馬番': cells[2].text.strip() if len(cells) > 2 else '',
                '馬名': cells[3].text.strip() if len(cells) > 3 else '',
                '性齢': cells[4].text.strip() if len(cells) > 4 else '',
                '斤量': cells[5].text.strip() if len(cells) > 5 else '',
                '騎手': jockey_val,
                'タイム': cells[7].text.strip() if len(cells) > 7 else '',
                '着差': cells[8].text.strip() if len(cells) > 8 else '',
                '人気': cells[9].text.strip() if len(cells) > 9 else '',\
                '単勝オッズ': cells[10].text.strip() if len(cells) > 10 else '',
                '後3F': cells[11].text.strip() if len(cells) > 11 else '',                'corner_1': '',
                'corner_2': '',
                'corner_3': '',
                'corner_4': '',
                '厩舎': stable_val,
                '調教師': trainer_val, # 新規追加
                '馬体重': weight_val,
                '増減': weight_change,
                'race_id': metadata['race_id'],
                'horse_id': ''
            }
            
            # 枠番(テキストから直接取得)
            if len(cells) > 1:
                horse_data['枠'] = cells[1].text.strip()
            
            # horse_id(リンクから抽出)
            horse_link = cells[3].find('a') if len(cells) > 3 else None
            if horse_link and 'href' in horse_link.attrs:
                horse_id_match = re.search(r'/horse/(\d+)', horse_link['href'])
                if horse_id_match:
                    horse_data['horse_id'] = horse_id_match.group(1)
            
            # コーナー通過順の個別カラム抽出
            corner_text = cells[12].text.strip() if len(cells) > 12 else ''
            if corner_text and '-' in corner_text:
                positions = corner_text.split('-')
                for j, pos in enumerate(positions[:4], 1):
                    horse_data[f'corner_{j}'] = pos.strip()
            
            race_data.append(horse_data)
        
        if not race_data:
            return None
        
        # DataFrameに変換(カラム順序を明示的に指定)
        ordered_columns = [
            '日付', '会場', 'レース番号', 'レース名', '重賞', 'コースタイプ', '距離', '回り',
            '天候', '馬場状態', '着順', '枠', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム',
            '着差', '人気', '単勝オッズ', '後3F', 'corner_1', 'corner_2', 'corner_3', 'corner_4',
            '厩舎', '調教師', '馬体重', '増減', 'race_id', 'horse_id'
        ]
        
        df = pd.DataFrame(race_data)[ordered_columns]
        return df
    
    except Exception as e:
        print(f"  ❌ エラー: {race_id} - {e}")
        return None

print("✅ Scraping function updated (with Trainer Fullname support)")


In [ ]:
# メインスクレイピング実行

def run_basic_scraping():
    """
    差分スクレイピング + チャンク保存
    """
    csv_path = os.path.join(SAVE_DIR, TARGET_CSV)
    master_path = os.path.join(SAVE_DIR, MASTER_ID_CSV)
    
    # マスターID読み込み
    if not os.path.exists(master_path):
        print(f"❌ マスターIDファイルが見つかりません: {master_path}")
        return
    
    master_df = pd.read_csv(master_path, dtype=str)
    master_ids = set(master_df['race_id'].dropna().unique())
    print(f"📋 マスターID数: {len(master_ids)}")
    
    # 既存ID読み込み
    existing_ids = set()
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path, dtype=str, usecols=['race_id'])
        existing_ids = set(existing_df['race_id'].dropna().unique())
        print(f"💾 既存ID数: {len(existing_ids)}")
    
    # 差分計算
    target_ids = sorted(list(master_ids - existing_ids))
    print(f"🚀 今回取得: {len(target_ids)} レース\n")
    
    if not target_ids:
        print("✅ 全て取得済みです")
        return
    
    # スクレイピング
    buffer = []
    chunk_size = 50
    
    for i, race_id in enumerate(tqdm(target_ids)):
        df = scrape_race_basic(race_id)
        
        if df is not None and not df.empty:
            buffer.append(df)
        
        # チャンク保存
        if len(buffer) >= chunk_size or (i == len(target_ids) - 1 and buffer):
            df_chunk = pd.concat(buffer, ignore_index=True)
            
            if not os.path.exists(csv_path):
                df_chunk.to_csv(csv_path, index=False)
            else:
                df_chunk.to_csv(csv_path, mode='a', header=False, index=False)
            
            print(f"  💾 Saved {len(buffer)} races")
            buffer = []
            gc.collect()
    
    print("\n✅ スクレイピング完了")

print("✅ Main function ready")

In [ ]:
# 実行
run_basic_scraping()